In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import h5py

In [ ]:
from environment.portfolio import PortfolioEnv

In [ ]:
eps=1e-8

In [ ]:
abbreviations = ['AAPL.O',
                 'ADBE.O',
                 'AMZN.O',
                 'DIS.N',
                 'GOOGL.O',
                 'JNJ.N',
                 'JPM.N',
                 'MSFT.O',
                 'NFLX.O',
                 'PG.N']

In [ ]:
with h5py.File('./Data/history_stock_price.h5','r') as f:
    history_stock_price = f['stock_price'][...]
    timestamp = [s.decode('utf-8') for s in f['timestamp']]
    abbreviations = [s.decode('utf-8') for s in f['abbreviations']]
    features = [s.decode('utf-8') for s in f['features']]

In [ ]:
env = PortfolioEnv(history=history_stock_price,abbreviation=abbreviations,timestamp=timestamp,steps=3000)

In [ ]:
obs, ground_truth_obs = env.reset()

In [ ]:
done = False
while not done:
    observation, onestep_r, done, info = env.step(np.random.rand(11))

In [ ]:
env.render()

In [ ]:
a = [1,2,3,4]

In [ ]:
a[-3:]

# DDPG

In [ ]:
tf.reset_default_graph()

In [ ]:
from model.ddpg.stockactor import StockActor
from model.ddpg.stockcritic import StockCritic
from model.ddpg.ddpg import DDPG
from model.core.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

In [ ]:
import json
with open('configs/ddpg_default.json') as f:
    config = json.load(f)

In [ ]:
actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(config['input']['asset_number']+1))

In [ ]:
sess = tf.Session()
stockactor = StockActor(sess, config, action_bound=1)
stockcritic = StockCritic(sess, config, num_actor_vars=stockactor.get_num_trainable_vars())

In [ ]:
myddpg = DDPG(env,sess,stockactor,critic=stockcritic,
              actor_noise=actor_noise, config=config)

In [ ]:
myddpg.initialize(load_weights=False)

In [ ]:
myddpg.train()

In [ ]:
pre_obs, ground_truth_obs = env.reset()
w = myddpg.predict(np.expand_dims(pre_obs,axis=0)).squeeze(axis=0)
for i in range(1000):
    obs, onestep_r, done, info = env.step(w)
    w = myddpg.predict(np.expand_dims(obs,axis=0)).squeeze(axis=0)

In [ ]:
env.render()